# Inclined alternating reinforcement

In [ ]:
%matplotlib widget
import sympy as sp
sp.init_printing()
import bmcs_utils.api as bu
import matplotlib.pylab as plt
import numpy as np

 - $ s_x$: Distance of intersections
 - $ \varphi$: Angle between beam direction and reinforcement inclination

In [ ]:
s_0_x = sp.Symbol(r's^0_x', positive=True)
phi = sp.Symbol(r'\varphi', positive=True)
eps_x = sp.Symbol(r'\varepsilon_x', positive=True)

$$ 
s^0_x = \cos({\varphi}) \; s^0_\varphi
$$
or inversely
$$
s^0_\varphi = \frac{s^0_x}{\cos \varphi}
$$

In [ ]:
s_0_phi_ = s_0_x / sp.cos(phi)
s_0_phi_

For a prescribed strain $\varepsilon_x$ we obtain
$$
  \Delta s_x = \varepsilon_x s^0_x
$$
So that the interim length of the cross-sectional segment reads
$$
 s^t_x = s^0_x + \Delta s_x
$$
rendering
$$
 s^t_x = s^0_x (1 + \varepsilon_x)
$$
 

In [ ]:
s_t_x_ = (1 + eps_x) * s_0_x
s_t_x_

$$
s_y = s^0_\varphi \; \sin \varphi
$$

In [ ]:
s_y_ = sp.simplify(s_0_phi_ * sp.sin(phi))
s_y_

$$
s^t_\varphi = \sqrt{ (s^t_x)^2 + (s^t_y)^2 }
$$

In [ ]:
s_t_phi_ = sp.sqrt( s_t_x_**2 + s_y_**2 )
s_t_phi_

$$
\varepsilon_\varphi = \frac{s^t_\varphi - s^0_\varphi}{s^0_\varphi}
$$

In [ ]:
eps_phi_ = sp.simplify( (s_t_phi_ - s_0_phi_) / s_0_phi_)
eps_phi_

In [ ]:
d_eps_phi_d_eps_x_ = sp.simplify(eps_phi_.diff(eps_x))

In [ ]:
B_ = sp.simplify(sp.limit( d_eps_phi_d_eps_x_, eps_x, 0) )
B_

In [ ]:
eps_phi_lin_ = B_ * eps_x
eps_phi_lin_

In [ ]:
get_eps_phi_ = sp.lambdify((eps_x, phi), eps_phi_)
get_eps_phi_lin_ = sp.lambdify((eps_x, phi), eps_phi_lin_)

In [ ]:
eps_x_range = np.linspace(0, 0.01, 40)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,3))
fig.canvas.header_visible = False
for phi_ in [0, np.pi/8, np.pi/4, 2*np.pi/5, np.pi/2]:
    eps_phi_range = get_eps_phi_(eps_x_range, phi_)
    ax.plot(eps_x_range, eps_phi_range);
    eps_phi_lin_range = get_eps_phi_lin_(eps_x_range, phi_)
    ax.plot(eps_x_range, eps_phi_lin_range);
#    ax.plot(eps_x_range[[0,-1]], eps_phi_range[[0,-1]])
ax.set_xlabel(r'$\varepsilon_x$ [-]');
ax.set_ylabel(r'$\varepsilon_\varphi$ [-]');

In [ ]:
eps_phi = sp.Symbol(r'\varepsilon_\phi')
f_t = sp.Symbol(r'f_\mathrm{t}', positive=True)
E = sp.Symbol(r'E', positive=True)
A = sp.Symbol(r'A', positive=True)
F_phi_ = sp.simplify(A * sp.Piecewise(
    (0, eps_phi < 0),
    (E * eps_phi, eps_phi < f_t / E ),
    (0, True)
))
get_F_phi_ = sp.lambdify((eps_phi, E, f_t), F_phi_)
F_phi_

In [ ]:
F_phi = sp.Symbol(r'F_\varphi')
F_x = sp.Symbol(r'F_x')

In [ ]:
get_B_ = sp.lambdify(phi, B_)
fig, ax = plt.subplots(1,1, figsize=(5,3))
fig.canvas.header_visible = False
phi_range = np.linspace(0, np.pi, 50)
B_range = get_B_(phi_range)
ax.plot(phi_range, B_range);
ax.plot(phi_range, np.cos(phi_range));
ax.set_xlabel(r'$\varepsilon_\phi$ [-]');
ax.set_ylabel(r'$B$ [-]');

In [ ]:
F_x_ = F_phi * B_
F_x_

In [ ]:
F_x__ = F_x_.subs(F_phi, F_phi_).subs(eps_phi, eps_x * B_)
get_F_x_ = sp.lambdify((eps_x, phi, E, A, f_t), F_x__)

In [ ]:
sp.simplify(F_x__)

In [ ]:
sp.simplify(F_x__.subs(eps_x, f_t / E))

In [ ]:
_E, _A, _f_t = 280000, 1, 3000 
eps_x_range = np.linspace(0, _f_t / _E * 3, 150)
fig, ax = plt.subplots(1,1, figsize=(5,3))
fig.canvas.header_visible = False
for phi_ in np.linspace(0, np.pi/2, 30):
    F_x_range = get_F_x_(eps_x_range, phi_, _E, _A, _f_t)
    ax.plot(eps_x_range, F_x_range);
ax.set_xlabel(r'$\varepsilon_x$ [-]');
ax.set_ylabel(r'$\varepsilon_\varphi$ [-]');